# Exploratory Data Analysis

In [4]:
#Setting the stage
#File path
data <- "/Users/patrickokwir/Desktop/Lighthouse-data-notes/Jack-Talk-Surveys/data_output/final_df_numeric.csv"

In [5]:
df <- read.csv(data)
head(df)

,id,Finished.x,RecordedDate.x,Q1_a,Q2_a,Q3_a,Q4_a,Pre,Finished.y,RecordedDate.y,Q1_b,Q2_b,Q3_b,Q4_b,Q5,Q6,Q7,Q8,Post,Matched
,<chr>,<lgl>,<chr>,<int>,<int>,<int>,<int>,<chr>,<lgl>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
1,A_A_01_A,TRUE,2023-05-15 14:05,6,6,6,6,pre,TRUE,2023-03-07 16:20,1,6,5,3,3,5,6,9,post,matched
2,A_A_01_A,TRUE,2022-11-21 17:38,5,2,5,5,pre,TRUE,2023-03-07 16:20,1,6,5,3,3,5,6,9,post,matched
3,A_A_01_C,TRUE,2023-05-01 12:21,3,2,5,3,pre,TRUE,2023-05-01 13:02,3,4,4,4,6,7,7,10,post,matched
4,A_A_01_D,TRUE,2023-05-02 14:13,6,5,5,3,pre,TRUE,2023-05-02 14:58,6,5,4,4,5,4,5,7,post,matched
5,A_A_01_M,TRUE,2023-03-29 20:42,6,7,5,3,pre,TRUE,2023-04-24 17:01,5,5,4,4,5,4,5,7,post,matched
6,A_A_01_M,TRUE,2023-04-24 16:19,3,5,4,3,pre,TRUE,2023-04-24 17:01,5,5,4,4,5,4,5,7,post,matched


In [8]:
# Find duplicates based on the 'id' column in df_final
duplicate_rows <- df[duplicated(df$id), ]

nrow(duplicate_rows)


[1] 785

In [12]:
library(dplyr)

# Arrange the data by 'id' and 'RecordedDate.y' in descending order
# Then, keep only the first occurrence of each unique 'id' and filter by RecordedDate.y >= RecordedDate.x
df <- df %>%
  arrange(id, desc(RecordedDate.y)) %>%
  group_by(id) %>%
  filter(RecordedDate.y >= RecordedDate.x) %>%
  slice_head(n = 1) %>%
  ungroup()

# Print the resulting data frame
head(df)
nrow(df)

id,Finished.x,RecordedDate.x,Q1_a,Q2_a,Q3_a,Q4_a,Pre,Finished.y,RecordedDate.y,Q1_b,Q2_b,Q3_b,Q4_b,Q5,Q6,Q7,Q8,Post,Matched
<chr>,<lgl>,<chr>,<int>,<int>,<int>,<int>,<chr>,<lgl>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>
A_A_01_A,TRUE,2022-11-21 17:38,5,2,5,5,pre,TRUE,2023-03-07 16:20,1,6,5,3,3,5,6,9,post,matched
A_A_01_C,TRUE,2023-05-01 12:21,3,2,5,3,pre,TRUE,2023-05-01 13:02,3,4,4,4,6,7,7,10,post,matched
A_A_01_D,TRUE,2023-05-02 14:13,6,5,5,3,pre,TRUE,2023-05-02 14:58,6,5,4,4,5,4,5,7,post,matched
A_A_01_M,TRUE,2023-03-29 20:42,6,7,5,3,pre,TRUE,2023-04-24 17:01,5,5,4,4,5,4,5,7,post,matched
A_A_02_A,TRUE,2023-05-01 13:45,7,6,5,5,pre,TRUE,2023-05-03 14:18,6,6,5,3,6,6,7,10,post,matched
A_A_02_C,TRUE,2023-03-26 15:06,6,6,6,6,pre,TRUE,2023-03-26 15:55,6,6,6,6,6,6,6,10,post,matched


[1] 2398

`Some of the post surveys had dates before the pre-surveys`

`5. Summarize the data for each of Q1-8 by reporting the mean response for each question. Please report the summary statistic for anyone who completed either the pre or post survey.`

In [13]:
library(dplyr)

# Calculate the mean response for each question separately
pre_survey_means <- df %>%
  filter(Pre == 'pre') %>%
  summarise(mean_Q1 = mean(Q1_a, na.rm = TRUE),
            mean_Q2 = mean(Q2_a, na.rm = TRUE),
            mean_Q3 = mean(Q3_a, na.rm = TRUE),
            mean_Q4 = mean(Q4_a, na.rm = TRUE))

post_survey_means <- df %>%
  filter(Post == 'post') %>%
  summarise(mean_Q1 = mean(Q1_b, na.rm = TRUE),
            mean_Q2 = mean(Q2_b, na.rm = TRUE),
            mean_Q3 = mean(Q3_b, na.rm = TRUE),
            mean_Q4 = mean(Q4_b, na.rm = TRUE),
            mean_Q5 = mean(Q5, na.rm = TRUE),
            mean_Q6 = mean(Q6, na.rm = TRUE),
            mean_Q7 = mean(Q7, na.rm = TRUE),
            mean_Q8 = mean(Q8, na.rm = TRUE))

pre_survey_means
post_survey_means

mean_Q1,mean_Q2,mean_Q3,mean_Q4
<dbl>,<dbl>,<dbl>,<dbl>
5.220183,4.479983,5.185154,4.78357


mean_Q1,mean_Q2,mean_Q3,mean_Q4,mean_Q5,mean_Q6,mean_Q7,mean_Q8
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5.297748,4.708924,5.312344,5.093411,5.183486,5.178482,5.024604,6.965388


In [22]:
# Combine pre and post survey means
combined_means <- bind_rows(
  mutate(pre_survey_means, Survey = "Pre"),
  mutate(post_survey_means, Survey = "Post")
)
combined_means

mean_Q1,mean_Q2,mean_Q3,mean_Q4,Survey,mean_Q5,mean_Q6,mean_Q7,mean_Q8
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
5.220183,4.479983,5.185154,4.783570,Pre,NA,NA,NA,NA
5.297748,4.708924,5.312344,5.093411,Post,5.183486,5.178482,5.024604,6.965388


`6. For individuals with both a pre and post response, compare the individual average change between the pre and post survey for each of Q1-4. Quantify the individual average change and report whether the difference between means is statistically significant.`

In [15]:
library(dplyr)
library(stats)

# Filter rows where both pre and post surveys were completed
completed_both <- df %>%
  filter(Pre == 'pre', Post == 'post') %>%
  select(id, starts_with("Q1"), starts_with("Q2"), starts_with("Q3"), starts_with("Q4"))

# Calculate the average change for each participant
completed_both <- completed_both %>%
  mutate(avg_change_Q1 = Q1_b - Q1_a,
         avg_change_Q2 = Q2_b - Q2_a,
         avg_change_Q3 = Q3_b - Q3_a,
         avg_change_Q4 = Q4_b - Q4_a)

# Calculate the mean and perform t-test for each question
result_summary <- completed_both %>%
  summarise(mean_change_Q1 = mean(avg_change_Q1),
            mean_change_Q2 = mean(avg_change_Q2),
            mean_change_Q3 = mean(avg_change_Q3),
            mean_change_Q4 = mean(avg_change_Q4),
            ttest_Q1 = t.test(avg_change_Q1)$p.value,
            ttest_Q2 = t.test(avg_change_Q2)$p.value,
            ttest_Q3 = t.test(avg_change_Q3)$p.value,
            ttest_Q4 = t.test(avg_change_Q4)$p.value)

# Print the summary of mean changes and t-test results
result_summary

mean_change_Q1,mean_change_Q2,mean_change_Q3,mean_change_Q4,ttest_Q1,ttest_Q2,ttest_Q3,ttest_Q4
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.07756464,0.2289408,0.1271893,0.3098415,0.00542085,1.84344e-17,1.060035e-07,1.686736e-34


In [17]:
library(dplyr)
library(stats)

# Filter rows where both pre and post surveys were completed
completed_both <- df %>%
  filter(Pre == 'pre', Post == 'post') %>%
  select(id, starts_with("Q1"), starts_with("Q2"), starts_with("Q3"), starts_with("Q4"))

# Calculate the average change for each participant
completed_both <- completed_both %>%
  mutate(avg_change_Q1 = Q1_b - Q1_a,
         avg_change_Q2 = Q2_b - Q2_a,
         avg_change_Q3 = Q3_b - Q3_a,
         avg_change_Q4 = Q4_b - Q4_a)

# Perform Wilcoxon signed-rank test for each question
result_summary <- completed_both %>%
  summarise(mean_change_Q1 = mean(avg_change_Q1),
            mean_change_Q2 = mean(avg_change_Q2),
            mean_change_Q3 = mean(avg_change_Q3),
            mean_change_Q4 = mean(avg_change_Q4),
            wilcox_Q1 = wilcox.test(avg_change_Q1)$p.value,
            wilcox_Q2 = wilcox.test(avg_change_Q2)$p.value,
            wilcox_Q3 = wilcox.test(avg_change_Q3)$p.value,
            wilcox_Q4 = wilcox.test(avg_change_Q4)$p.value)

# Print the summary of Wilcoxon signed-rank test results
result_summary

mean_change_Q1,mean_change_Q2,mean_change_Q3,mean_change_Q4,wilcox_Q1,wilcox_Q2,wilcox_Q3,wilcox_Q4
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.07756464,0.2289408,0.1271893,0.3098415,0.002943773,2.123835e-18,9.378653e-08,2.670837e-37


`Lets explore Ordinal Regression: model an ordinal regression to analyze the relationship between predictor variables (change in q1 to q4 because it has pre and post pairs), q5-q7 and the ordinal response Q8 (would they reccomend the program).`

In [37]:
df_final <- df

library(MASS)
library(dplyr)

#polr requires the dependent variable to be a factor
df_final$Q8 <- as.factor(df_final$Q8)

# Calculate average change for each question
df_final$avg_change_Q1 <- df_final$Q1_b - df_final$Q1_a
df_final$avg_change_Q2 <- df_final$Q2_b - df_final$Q2_a
df_final$avg_change_Q3 <- df_final$Q3_b - df_final$Q3_a
df_final$avg_change_Q4 <- df_final$Q4_b - df_final$Q4_a

# Fit the ordinal regression model
model <- polr(Q8 ~ avg_change_Q1 + avg_change_Q2 + avg_change_Q3 + avg_change_Q4 + Q5 + Q6 + Q7, data = df_final, method = "logistic")

# Print the model summary
print(model)


Call:
polr(formula = Q8 ~ avg_change_Q1 + avg_change_Q2 + avg_change_Q3 + 
    avg_change_Q4 + Q5 + Q6 + Q7, data = df_final, method = "logistic")

Coefficients:
avg_change_Q1 avg_change_Q2 avg_change_Q3 avg_change_Q4            Q5 
  -0.01178078    0.02885824    0.03110088   -0.05236186    0.31917856 
           Q6            Q7 
   0.30071655    0.58818291 

Intercepts:
     0|1      1|2      2|3      3|4      4|5      5|6      6|7      7|8 
1.414372 1.711318 2.136753 2.569527 3.052560 4.854238 5.454709 6.549326 
     8|9     9|10 
7.739787 8.512687 

Residual Deviance: 8553.739 
AIC: 8587.739 


`Coefficients: These are the estimated coefficients for each predictor variable in the model.`

`avg_change_Q1, avg_change_Q2, avg_change_Q3, and avg_change_Q4: These are the coefficients associated with the average change in Q1, Q2, Q3, and Q4, respectively. The negative sign for avg_change_Q1 and avg_change_Q4 suggests that an increase in the average change of these questions is associated with a lower likelihood of a higher response in Q8 (ordinal response).`

`Q5, Q6, and Q7: These coefficients are associated with the predictor variables Q5, Q6, and Q7. For example, a positive coefficient for Q5 suggests that a higher value in Q5 is associated with a higher likelihood of a higher response in Q8.`